# Study 2

In [1]:
from sklearn.datasets import make_blobs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn import metrics
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [10]:
data = pd.read_csv("data_study2.csv")
data

,TI,EES,JIMS,SPOS,BFI,ACS,NCS,MOAQ,GSOI,BIO1,...,BIO4,BIO5,BIO6,BIO7,BIO8,BIO9_1,BIO10,BIO11,BIO12,BIO13
0,0,3.666667,3.1,2.500000,2.666667,3.000000,3.000,3.000000,4,4,...,2,3,3,2,3,3.11,5,3,3,3
1,0,4.000000,1.8,5.000000,4.833333,2.888889,1.250,5.000000,1,5,...,2,5,1,2,1,3.53,5,5,5,5
2,0,3.666667,2.9,4.333333,3.833333,3.555556,3.500,5.000000,4,3,...,2,3,2,1,3,3.00,3,5,5,5
3,1,3.666667,3.1,3.166667,3.166667,2.555556,3.125,3.000000,3,3,...,2,4,2,3,3,2.50,3,2,3,5
4,0,3.666667,3.3,4.166667,4.166667,2.777778,3.000,3.666667,5,1,...,1,3,1,1,1,3.93,5,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,2.000000,2.3,1.333333,3.833333,3.222222,2.750,4.000000,3,1,...,2,4,1,1,1,2.99,3,3,1,5
237,0,4.000000,2.9,3.833333,3.833333,2.888889,2.250,4.000000,3,1,...,2,5,2,4,5,3.74,6,3,3,5
238,0,4.000000,2.4,5.000000,3.000000,3.555556,2.500,4.666667,7,3,...,3,5,2,1,2,3.00,3,3,3,5
239,0,3.000000,2.1,2.333333,1.833333,2.777778,1.875,3.666667,4,4,...,2,5,2,5,1,4.00,8,3,2,5


# Baseline models: biodata only

In [11]:
from sklearn.model_selection import train_test_split
y=data['TI']
X=data.iloc[:,9:22]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)



In [12]:
from sklearn.linear_model import LogisticRegression
#X_train = np.array(X_train).reshape(-1,1)
#y_train = np.array(y_train).reshape(-1,1)
#X_test = np.array(X_test).reshape(-1,1)
#y_test = np.array(y_test).reshape(-1,1)
reg = LogisticRegression().fit(X_train, y_train)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
# For AUC-ROC, you need the probabilities of the positive class
y_pred_logi = reg.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_logi = roc_auc_score(y_test, y_pred_logi)
print(f"AUC-ROC: {auc_roc_logi}")

AUC-ROC: 0.6866028708133971


In [13]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



param_grid = {'max_depth':range(1, 5),
                'learning_rate':(0.005, 0.05, 0.5)}
 
grid = GridSearchCV(xgb.XGBClassifier(random_state=0), 
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')
    
grid.fit(X_train, y_train)
# For AUC-ROC, you need the probabilities of the positive class
y_pred_xgb = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_xgb = roc_auc_score(y_test, y_pred_xgb)
print(f"AUC-ROC: {auc_roc_xgb}")



Fitting 10 folds for each of 12 candidates, totalling 120 fits
AUC-ROC: 0.6267942583732057


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
import numpy as np

# Define a simple neural network model for binary classification
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with optimizer, loss function, and metrics for binary classification
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
pred_probs = model.predict(X_test)
pred_nn = np.round(pred_probs).astype(int)  # Convert probabilities to binary class labels

# Computing AUC-ROC
auc_roc_nn = roc_auc_score(y_test, pred_probs)
print(f"AUC-ROC: {auc_roc_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7144 - loss: 0.6769
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7021 - loss: 0.5946 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7169 - loss: 0.5499 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7557 - loss: 0.5148 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7254 - loss: 0.5221 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7549 - loss: 0.4911 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7712 - loss: 0.4941 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7531 - loss: 0.5041 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7696 - loss: 0.4835 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8249 - loss: 0.4383 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
AUC-ROC: 0.7296650717703349


In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('clf', LinearSVC(dual=False))  # 'clf' is the name we give to the LinearSVC step
])



# Define a very simple parameter grid, could be expanded based on need
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__max_iter': [1000, 2000, 3000],
    'clf__intercept_scaling': [1, 10, 100]
}


# Initialize GridSearchCV with the SVC estimator and the simple param grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svm = grid.predict(X_test)

# Computing AUC-ROC
auc_roc_svm = roc_auc_score(y_test, pred_svm)
print(f"AUC-ROC: {auc_roc_svm}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.001;, score=0.700 total time=   0.0s
[CV 2/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.001;, score=0.700 total time=   0.0s
[CV 3/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.001;, score=0.684 total time=   0.0s
[CV 4/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.001;, score=0.737 total time=   0.0s
[CV 5/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.001;, score=0.737 total time=   0.0s
[CV 6/10] END clf__C=0.1, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.00

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adjusting the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')

# Fit to the training data
grid.fit(X_train, y_train)

# For AUC-ROC, you need the probabilities of the positive class
y_pred_rf = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC: {auc_roc_rf}")

Fitting 10 folds for each of 16 candidates, totalling 160 fits


C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

AUC-ROC: 0.6578947368421052


C:\Users\diego\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [40]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Logistic Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "AUC Score": [auc_roc_logi, auc_roc_xgb, auc_roc_nn, auc_roc_svm, auc_roc_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "base_model_auc_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)

# WRC Model: work-related psychological constructs only

In [58]:
from sklearn.model_selection import train_test_split
y=data['TI']
X=data[['EES', 'JIMS', 'SPOS', 'BFI', 'ACS', 'NCS', 'MOAQ', 'GSOI']]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)

In [59]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X_train, y_train)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
# For AUC-ROC, you need the probabilities of the positive class
y_pred_logi = reg.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_logi = roc_auc_score(y_test, y_pred_logi)
print(f"AUC-ROC: {auc_roc_logi}")

AUC-ROC: 0.8086124401913876


In [60]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



param_grid = {'max_depth':range(1, 5),
                'learning_rate':(0.005, 0.05, 0.5)}
 
grid = GridSearchCV(xgb.XGBClassifier(random_state=0), 
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')
    
grid.fit(X_train, y_train)
# For AUC-ROC, you need the probabilities of the positive class
y_pred_xgb = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_xgb = roc_auc_score(y_test, y_pred_xgb)
print(f"AUC-ROC: {auc_roc_xgb}")


Fitting 10 folds for each of 12 candidates, totalling 120 fits
AUC-ROC: 0.7763157894736842


In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
import numpy as np

# Define a simple neural network model for binary classification
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with optimizer, loss function, and metrics for binary classification
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
pred_probs = model.predict(X_test)
pred_nn = np.round(pred_probs).astype(int)  # Convert probabilities to binary class labels

# Computing AUC-ROC
auc_roc_nn = roc_auc_score(y_test, pred_probs)
print(f"AUC-ROC: {auc_roc_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4402 - loss: 0.7960  
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7414 - loss: 0.5439 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7026 - loss: 0.5917 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7422 - loss: 0.5022 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7044 - loss: 0.5439 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7189 - loss: 0.5227 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7184 - loss: 0.5136 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7659 - loss: 0.4616 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7525 - loss: 0.4692 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8142 - loss: 0.4191 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
AUC-ROC: 0.7392344497607655


In [62]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('clf', LinearSVC(dual=False))  # 'clf' is the name we give to the LinearSVC step
])



# Define a very simple parameter grid, could be expanded based on need
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__max_iter': [1000, 2000, 3000],
    'clf__intercept_scaling': [1, 10, 100]
}


# Initialize GridSearchCV with the SVC estimator and the simple param grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svm = grid.predict(X_test)

# Computing AUC-ROC
auc_roc_svm = roc_auc_score(y_test, pred_svm)
print(f"AUC-ROC: {auc_roc_svm}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.0001;, score=0.750 total time=   0.0s
[CV 2/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.0001;, score=0.650 total time=   0.0s
[CV 3/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.0001;, score=0.895 total time=   0.0s
[CV 4/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.0001;, score=0.895 total time=   0.0s
[CV 5/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.0001;, score=0.789 total time=   0.0s
[CV 6/10] END clf__C=100, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adjusting the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')

# Fit to the training data
grid.fit(X_train, y_train)

# For AUC-ROC, you need the probabilities of the positive class
y_pred_rf = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC: {auc_roc_rf}")

Fitting 10 folds for each of 16 candidates, totalling 160 fits
AUC-ROC: 0.7703349282296651


In [64]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Logistic Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "AUC Score": [auc_roc_logi, auc_roc_xgb, auc_roc_nn, auc_roc_svm, auc_roc_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "addedWRC_model_auc_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)

# All-variables model

In [68]:
from sklearn.model_selection import train_test_split
y=data['TI']
X=data.iloc[: , 1:]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)

In [69]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X_train, y_train)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
# For AUC-ROC, you need the probabilities of the positive class
y_pred_logi = reg.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_logi = roc_auc_score(y_test, y_pred_logi)
print(f"AUC-ROC: {auc_roc_logi}")

AUC-ROC: 0.8110047846889953


In [70]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



param_grid = {'max_depth':range(1, 5),
                'learning_rate':(0.005, 0.05, 0.5)}
 
grid = GridSearchCV(xgb.XGBClassifier(random_state=0), 
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')
    
grid.fit(X_train, y_train)
# For AUC-ROC, you need the probabilities of the positive class
y_pred_xgb = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_xgb = roc_auc_score(y_test, y_pred_xgb)
print(f"AUC-ROC: {auc_roc_xgb}")


Fitting 10 folds for each of 12 candidates, totalling 120 fits
AUC-ROC: 0.7763157894736842


In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
import numpy as np

# Define a simple neural network model for binary classification
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with optimizer, loss function, and metrics for binary classification
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
pred_probs = model.predict(X_test)
pred_nn = np.round(pred_probs).astype(int)  # Convert probabilities to binary class labels

# Computing AUC-ROC
auc_roc_nn = roc_auc_score(y_test, pred_probs)
print(f"AUC-ROC: {auc_roc_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5951 - loss: 0.6097  
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7235 - loss: 0.5483 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7114 - loss: 0.5184 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7640 - loss: 0.4821 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7157 - loss: 0.5043 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7591 - loss: 0.4543 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7467 - loss: 0.5023 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7743 - loss: 0.4629 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7963 - loss: 0.4369 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8216 - loss: 0.4102 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AUC-ROC: 0.7440191387559808


In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('clf', LinearSVC(dual=False))  # 'clf' is the name we give to the LinearSVC step
])



# Define a very simple parameter grid, could be expanded based on need
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__max_iter': [1000, 2000, 3000],
    'clf__intercept_scaling': [1, 10, 100]
}


# Initialize GridSearchCV with the SVC estimator and the simple param grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svm = grid.predict(X_test)

# Computing AUC-ROC
auc_roc_svm = roc_auc_score(y_test, pred_svm)
print(f"AUC-ROC: {auc_roc_svm}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.001;, score=0.650 total time=   0.0s
[CV 2/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.001;, score=0.700 total time=   0.0s
[CV 3/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.001;, score=0.789 total time=   0.0s
[CV 4/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.001;, score=0.789 total time=   0.0s
[CV 5/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.001;, score=0.737 total time=   0.0s
[CV 6/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__to

[CV 2/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.700 total time=   0.0s
[CV 3/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.789 total time=   0.0s
[CV 4/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.842 total time=   0.0s
[CV 5/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.789 total time=   0.0s
[CV 6/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.789 total time=   0.0s
[CV 7/10] END clf__C=0.01, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l2, clf__tol=0.01;, score=0.895 total time=   0.0s
[CV 8/10] END cl

C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\diego\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number 

In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adjusting the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')

# Fit to the training data
grid.fit(X_train, y_train)

# For AUC-ROC, you need the probabilities of the positive class
y_pred_rf = grid.predict_proba(X_test)[:, 1]

# Computing AUC-ROC
auc_roc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC: {auc_roc_rf}")

Fitting 10 folds for each of 16 candidates, totalling 160 fits
AUC-ROC: 0.7787081339712918


In [74]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Logistic Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "AUC Score": [auc_roc_logi, auc_roc_xgb, auc_roc_nn, auc_roc_svm, auc_roc_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "ALL_model_auc_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)